In [1]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
verify_few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/Food-verify.expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel-ner-qa'
model_name = "gpt-3.5-turbo"
label_column = 'Food_answer'
text_column = 'context'
explanation_column = 'Food_explanation'
example_selection_label_column = 'has_Food'
label_symbol = "^^^^"
few_shot_num = 8
few_shot_selection = 'label_diversity_similarity'
verify_few_shot_selection = "semantic_similarity"
use_current_explanation = False
use_ground_explanation = False
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=True
temperature=0.1
verify=True
label_version='v1'
task_version='v2'
random_shuffle_examples = True
random_shuffle_examples_seed = 1

# Generation Stage

In [2]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from question_answering_new import NewQuestionAnsweringTask
from attribute_extraction_new import NewAttributeExtractionTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION
from prompt_template import load_ner_ae_prompt
from dataset_new import process_labels
import sys

In [3]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.QUESTION_ANSWERING] = NewQuestionAnsweringTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.ATTRIBUTE_EXTRACTION] = NewAttributeExtractionTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
sys.modules['autolabel.dataset'].AutolabelDataset.process_labels = process_labels
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [4]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [5]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [6]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [7]:
label_type = label_column.split('_')[0]
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_AE_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [8]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERAE_{few_shot_selection}_{label_type}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERAE_{few_shot_selection}_{label_type}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetNERAE_{few_shot_selection}_{label_type}_{model_name.split('/')[1]}"

In [9]:
sym_len = len(label_symbol)
label_prefix, label_suffix, label_description, task_guideline, output_guideline = load_ner_ae_prompt(label_symbol, label_version, task_version, True)

In [10]:
test_data = pd.read_csv(data_path)

# Verification Stage

In [11]:
last_results = pd.read_csv(output_path) 

In [12]:
from utils import get_predictions, check_prediction, get_verify_df, construct_results

In [13]:
verify_question_column = f'{label_type}_verify_question'
verify_answer_column = f'{label_type}_verify_answer'
verify_explanation_column = f'{label_type}_verify_explanation'
pos_column = f'{label_column}_pos'

In [14]:
test_data[[verify_question_column, pos_column]] = last_results[[f'{label_column}_label']].apply(lambda x: get_predictions(x[f'{label_column}_label'], label_symbol), axis=1,result_type="expand")

In [15]:
test_data[verify_answer_column] = test_data[['CategorizedLabels', verify_question_column]].apply(lambda x: check_prediction(x['CategorizedLabels'], x[verify_question_column], label_type), axis=1)

In [16]:
new_test_data = get_verify_df(test_data, verify_question_column, verify_answer_column, text_column, pos_column)

In [17]:
verify_task_guideline = (f"You are an expert at identifying {label_type} entities that are related to foodborne illness incident from text. " 
                         f"In the given text, your task is to verify if a given word is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')} in the given text. "
                         "Use the following examples as a guide for your analysis and format your responses similarly.")
question = f'''Do you think the word "{{{verify_question_column}}}" in the given text is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')}?'''

In [18]:
if verify_few_shot_selection == few_shot_selection:
    output_verify_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_AE_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + label_type + '_verify_COT_' + os.path.split(data_path)[-1]
    output_verify_path = os.path.join(output_dir, output_verify_name)
    output_final_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_AE_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + label_type + '_final_COT_' + os.path.split(data_path)[-1]
else:
    output_verify_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_AE_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + label_type + '_verify_COT_' + verify_few_shot_selection + '_' + os.path.split(data_path)[-1]
    output_verify_path = os.path.join(output_dir, output_verify_name)
    output_final_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_AE_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + label_type + '_final_COT_' + verify_few_shot_selection + '_' + os.path.split(data_path)[-1]
output_final_path = os.path.join(output_dir, output_final_name)

In [19]:
config = {
    "task_name": task_name+'_verification',
    "task_type": "classification",
    "dataset": {
        "label_column": verify_answer_column,
        "text_column": text_column,
        "explanation_column": verify_explanation_column,
        "example_selection_label_column": example_selection_label_column,
        "delimiter": ",",
        "label_description": label_description
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": verify_task_guideline,
        "output_guidelines": ('''Your answer will consist of an explanation, followed by the correct answer ("Yes" or "No").'''
                              f'''Please answer with "Yes" if the given word is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')} in the given text, otherwise answer with "No".'''
                              '''The last line of the response should always be JSON format with one key: {"label": "the correct answer"}.\n'''
                             ),
        "labels": [
            "Yes",
            "No"
        ],
        "example_selection_labels":[
            "yes",
            "no"
        ],
        "few_shot_examples": verify_few_shot_path,
        "few_shot_selection": verify_few_shot_selection,
        "few_shot_num": few_shot_num,
        "use_current_explanation": use_current_explanation,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Context: {{{text_column}}}\nQuestion: {question}\nAnswer: Let's think step by step.\n{{{verify_explanation_column}}}\n{{{verify_answer_column}}}",
        "chain_of_thought": True
    }
}

config = NewAutoLabelConfig(config)

In [20]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-03-20 23:54:59 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [21]:
verify_ds = AutolabelDataset(new_test_data.drop(['text_idx',pos_column], axis=1), config=config)

In [22]:
agent.plan(verify_ds)

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-03-20 23:55:02 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:55:07 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:55:11 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Output()

2024-03-20 23:55:14 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:55:14 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:55:14 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:55:15 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:55:15 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:55:15 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:55:15 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $0.0681 │
│ Number of Examples       │ 7       │
│ Average cost per example │ $0.0097 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at identifying Food entities that are related to foodborne illness incident from text. In the 
given text, your task is to verify if a given word is a Food entity that is specific food item that caused the 
potential foodborne illness incident. in the given text. Use the following examples as a guide for your analysis 
and format your responses similarly.

Your answer will consist of an explanation, followed by the correct answer ("Yes" or "No").Please answer with "Yes"
if the given word is a Food entity that is specific food item that caused the potential foodborne illness incident.
in the given text, otherwise answer with "No".The last line of the response should always be JSON format with one 
key: {"label": "the correct answer"}.


Some examples with their output answers are provided below:

Context: I am either dying or have food poisoning . . . I'm going with food poisoning since I cooked meat today [ 
EMOJI_face_with_medical_mask ]
Question: Do you think the word meat in the given text is a Food entity that is specific food item that caused the 
potential foodborne illness incident.?
Answer: Let's think step by step.
The word "meat" is a specific food item that caused the potential foodborne illness incident.
{"label": "Yes"}
Context: @USER Sure its food poisoning ? Really bad stomach flu going around the whole country . Feels like food 
poisoning .
Question: Do you think the word poisoning ? Really in the given text is a Food entity that is specific food item 
that caused the potential foodborne illness incident.?
Answer: Let's think step by step.
The word "poisoning ? Really" does not cause a potential foodborne illness incident.
{"label": "No"}
Context: Think I might have gotten a mild case of food-poisoning today - so that's been super fun .
Question: Do you think the word gotten in the given text is a Food entity that is specific food item that caused 
the potential foodborne illness incident.?
Answer: Let's think step by step.
The word "gotten" does not cause a potential foodborne illness incident.
{"label": "No"}
Context: @USER LOL I know ! I've had a stomach bug or food poisoning since yesterday afternoon
Question: Do you think the word stomach bug or in the given text is a Food entity that is specific food item that 
caused the potential foodborne illness incident.?
Answer: Let's think step by step.
The word "stomach bug or" does not cause a potential foodborne illness incident.
{"label": "No"}
Context: @USER @USER @USER LOL DONT GET FOOD POISONING ! Me and my friend both got a taco in a bag on Tuesday and 
we were both extremely sick the next morning
Question: Do you think the word taco in the given text is a Food entity that is specific food item that caused the 
potential foodborne illness incident.?
Answer: Let's think step by step.
The word "taco" is a specific food item that caused the potential foodborne illness incident.
{"label": "Yes"}
Context: I've had food poisoning all day and I'm just miserable
Question: Do you think the word I've had food in the given text is a Food entity that is specific food item that 
caused the potential foodborne illness incident.?
Answer: Let's think step by step.
The word "I've had food" does not cause a potential foodborne illness incident.
{"label": "No"}
Context: @USER @USER I ate wing fries and got food poisoning bro . I do not wish this feeling upon anyone . I've 
been peeing out my ass for the past 12 hours or so
Question: Do you think the word wing fries in the given text is a Food entity that is specific food item that 
caused the potential foodborne illness incident.?
Answer: Let's think step by step.
The word "wing fries" is a specific food item that caused the potential foodborne illness incident.
{"label": "Yes"}
Context: All that free food yesterday made me SO sick [ EMOJI_face_with_tears_of_joy ] idk if I got a bit of food 
poisoning or if a random 24 hour bug suddenly came on , but WOW it was intense !
Question: Do you think the word free 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [23]:
# now, do the actual labeling
verify_ds = agent.run(verify_ds)

Output()

2024-03-20 23:45:53 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:45:54 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-20 23:45:54 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:45:55 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-20 23:45:55 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:45:56 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-20 23:45:56 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:45:57 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-03-20 23:45:58 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-20 23:45:58 ht

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classification_report:
              precision    recall  f1-score   support

          No       1.00      0.57      0.73         7
         Yes       0.00      0.00      0.00         0

    accuracy                           0.57         7
   macro avg       0.50      0.29      0.36         7
weighted avg       1.00      0.57      0.73         7

Actual Cost: 0.0111

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.5714   │ 7       │ 1.0             │
└──────────┴─────────┴─────────────────┘

In [31]:
metrics = verify_ds.eval()

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃ classification_report                                 ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0.6667   │ 9       │ 1.0             │               precision    recall  f1-score   support │
│          │         │                 │                                                       │
│          │         │                 │           No       1.00      0.67      0.80         9 │
│          │         │                 │          Yes       0.00      0.00      0.00         0 │
│          │         │                 │                                                       │
│          │         │                 │     accuracy                           0.67         9 │
│          │         │                 │    macro avg       0.50      0.33      0.40         9 │
│          │         │                 │ weighted avg       1.00      0.67      0.80         9 │
│          │         │                 │                                                       │
└──────────┴─────────┴─────────────────┴───────────────────────────────────────────────────────┘

In [ ]:
new_test_data[f'{verify_answer_column}_label'] = verify_ds.df[f'{verify_answer_column}_label']

In [ ]:
verify_ds.df['text_idx'] = new_test_data['text_idx']

In [ ]:
verify_ds.df.to_csv(output_verify_path, index=False)
verify_ds.df.to_pickle(output_verify_path.replace('.csv', '.pkl'))

In [ ]:
new_result = construct_results(last_results, new_test_data, f'{label_column}_label', f'{verify_answer_column}_label', "text_idx", pos_column, label_symbol)

In [ ]:
new_result.to_csv(output_final_path, index=False)
new_result.to_pickle(output_final_path.replace('.csv', '.pkl'))